In [2]:
import org.apache.spark.sql.types.{StructType, StructField, IntegerType, LongType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame

lastException: Throwable = null


In [3]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.4


In [4]:
val inputStreamPath = "/home/jovyan/data/events-stream"
val modelPath = "/home/jovyan/models/spark-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

inputStreamPath = /home/jovyan/data/events-stream
modelPath = /home/jovyan/models/spark-ml-model
dataSchema = StructType(StructField(tweet,StringType,true))
inputDF = [tweet: string]


[tweet: string]

In [5]:
/*
inputDF.writeStream
      .format("console")
      .outputMode("append")
      .start()             // Start the computation
      .awaitTermination()  // Wait for the computation to terminate

*/

Name: Syntax Error.
Message: 
StackTrace: 

In [6]:

import java.time.Instant

val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))
val getTimestamp = udf(() => Instant.now.getEpochSecond )

val loadedModel = PipelineModel.load("/home/jovyan/models/spark-ml-model")

val dataSchema = new StructType()
    .add("tweet", StringType)
    .add("prediction", IntegerType)
    .add("clean_probability", LongType)
    .add("TS", LongType)

var predDF = Seq.empty[(String, Int, Long, Long)].toDF("tweet", "prediction", "clean_probability", "TS")

inputDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    
    val predictionsDF = loadedModel.transform(batchDF)
    val now_minus_10 = getTimestamp() - 10
    val resDF = predictionsDF.select($"tweet", $"prediction", getProbability($"probability").alias("clean_probability"), getTimestamp().alias("TS"))
        
    predDF = predDF.union(resDF)
    //resDF.show(1000)
    println(predDF.count()) //.filter($"TS" >= now_minus_10)    
    //predDF.filter($"TS" >= now_minus_10).groupBy("prediction").count().show()
}.start()

15
33


getProbability = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))
getTimestamp = UserDefinedFunction(<function0>,LongType,Some(List()))
loadedModel = pipeline_d0f6df2ac549
dataSchema = StructType(StructField(tweet,StringType,true), StructField(prediction,IntegerType,true), StructField(clean_probability,LongType,true), StructField(TS,LongType,true))
predDF = [tweet: string, prediction: int ... 2 more fields]


org.apache.spark.sql.execution.streamin...

47
57
73
86
91
106
127
137
